In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

Mon Nov 29 21:18:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
torch.cuda.is_available()

True

In [6]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 39.7 MB/s 
     |████████████████████████████████| 895 kB 39.0 MB/s 
     |████████████████████████████████| 3.3 MB 25.0 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss
import torch.nn.functional as F

import torch.optim as optim
from torchtext.legacy.data import BucketIterator

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

In [7]:
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    get_linear_schedule_with_warmup,
    BertConfig,
    BertModel,
    BertPreTrainedModel,
    BertTokenizer,)

In [8]:
from transformers import glue_convert_examples_to_features as convert_examples_to_features
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers.data.processors.utils import InputExample, DataProcessor

In [9]:
import logging
logger=logging.getLogger(__name__)
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
MODEL_CLASSES={
    "bert":(BertConfig,BertTokenizer),
}

my_label_list=[0, 1]
MAX_SEQ_LENGTH=200

In [ ]:
pwd

'/content'

In [11]:
with open('/content/drive/MyDrive/bert/MLProject-bert/bert-master/dataset/train_en.json','r',encoding = 'UTF-8') as f:
  data = json.load(f)
  f.close()
print("File is successfully loaded")

File is successfully loaded


In [ ]:
!pip install torch


In [12]:
pd.DataFrame(data).to_csv('train_en.csv',index =False)
train_data = pd.read_csv('train_en.csv')
train_data.head()

,content,label
0,The number of people affected is as follows:\n...,1
1,Macau detects the second case of coronavirus i...,1
2,FACTBOX-Sports events hit by the coronavirus e...,1
3,Fight against Coronavirus: Italy seals off the...,1
4,stranded film crew made a chilling short docu ...,1


In [13]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
train_data['label'] = LE.fit_transform(train_data['label'])
train_data.head()

,content,label
0,The number of people affected is as follows:\n...,1
1,Macau detects the second case of coronavirus i...,1
2,FACTBOX-Sports events hit by the coronavirus e...,1
3,Fight against Coronavirus: Italy seals off the...,1
4,stranded film crew made a chilling short docu ...,1


In [14]:
train = train_data.copy()
train = train.reindex(np.random.permutation(train.index))
train.head()

,content,label
381,Global economy already in recession on coronav...,1
211,health minister asks pofma office to issue cor...,1
1492,There's a new suspicious case of infection wit...,1
1614,Coronavirus FAQs: All your questions about Cov...,1
1578,Wuhan closes makeshift hospital as new coronav...,1


In [15]:
import re
def clean_txt(text):
  text = re.sub("'", "",text)
  text=re.sub("(\\W)+"," ",text)    
  return text

In [16]:
for i in range(len(train)):
  train['content'][i] = clean_txt(train['content'][i])
train.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,content,label
381,Global economy already in recession on coronav...,1
211,health minister asks pofma office to issue cor...,1
1492,Theres a new suspicious case of infection with...,1
1614,Coronavirus FAQs All your questions about Covi...,1
1578,Wuhan closes makeshift hospital as new coronav...,1


In [17]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=35)
train.head()

,content,label
1148,Number of casualties due to new coronavirus go...,1
1003,YouTube pulled 14 videos with fake medical tre...,1
224,Covid 19 Penafiel and Amarante hospitals impos...,1
1758,Coronavirus China reports 27 new deaths China ...,1
1180,Deeply concerned WHO declares coronavirus outb...,0


In [18]:
train.reset_index(drop=True, inplace=True)
train.head(2)

,content,label
0,Number of casualties due to new coronavirus go...,1
1,YouTube pulled 14 videos with fake medical tre...,1


In [19]:
val.reset_index(drop=True, inplace=True)
val.head(2)

,content,label
0,Coronavirus crisis Indias toy industry reels u...,1
1,India is well stocked with medicines to contai...,1


In [20]:
val.shape, train.shape

((440, 2), (1760, 2))

按照每200大小切割文本

In [21]:
def get_split(text):
    l_total = []
    l_parcial = []
    if len(text.split())//150 >0:
      n = len(text.split())//150
    else: 
      n = 1
    for w in range(n):
      if w == 0:
        l_parcial = text.split()[:200]
        l_total.append(" ".join(l_parcial))
      else:
        l_parcial = text.split()[w*150:w*150 + 200]
        l_total.append(" ".join(l_parcial))
    return l_total

In [22]:
train['text_split'] = train['content'].apply(get_split)
train.head()
train_copy = train

In [23]:
val['text_split'] = val['content'].apply(get_split)
val.head()

,content,label,text_split
0,Coronavirus crisis Indias toy industry reels u...,1,[Coronavirus crisis Indias toy industry reels ...
1,India is well stocked with medicines to contai...,1,[India is well stocked with medicines to conta...
2,Indonesia says nothing to hide after no corona...,1,[Indonesia says nothing to hide after no coron...
3,Parliament is debating Wednesday on the preven...,1,[Parliament is debating Wednesday on the preve...
4,FOREX Dollar shines against euro riskier peers...,1,[FOREX Dollar shines against euro riskier peer...


In [24]:
train_l = []  # 分割好的文本
label_l = []  # 每段文本的label
index_l =[]   # 该段文本属于未分割前的哪个文本. 比如是3, 那就表明此段文本是属于未分割前的第4条数据
for idx,row in train.iterrows():
  for l in row['text_split']:
    train_l.append(l)
    label_l.append(row['label'])
    index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(6570, 6570, 6570)

In [25]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val.iterrows():
  for l in row['text_split']:
    val_l.append(l)
    val_label_l.append(row['label'])
    val_index_l.append(idx)
len(val_l), len(val_label_l), len(val_index_l)

(1670, 1670, 1670)

In [26]:
train_df = pd.DataFrame({'content':train_l, 'label':label_l})
train_df.head()

,content,label
0,Number of casualties due to new coronavirus go...,1
1,from a two month stay in Wuhan The confirmed c...,1
2,have been cancelled including in Macao At leas...,1
3,capital of the game may be forced to close if ...,1
4,provide isolated and efficient treatment for i...,1


In [27]:
val_df = pd.DataFrame({'content':val_l, 'label':val_label_l})
val_df.head()

,content,label
0,Coronavirus crisis Indias toy industry reels u...,1
1,crisis China factories were shut for New Year ...,1
2,its heavy dependence on China is more vulnerab...,1
3,Coronavirus in China Nijhawan said Traders kno...,1
4,the sales tremendously and adversely affecting...,1


In [28]:
train_InputExamples = train_df.apply(lambda x: InputExample(guid=None,
                                                                   text_a = x['content'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

val_InputExamples = val_df.apply(lambda x: InputExample(guid=None, 
                                                                   text_a = x['content'], 
                                                                   text_b = None, 
                                                                   label = x['label']), axis = 1)

定义类型

In [29]:
class BertForClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 10

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,  #输入的id,模型会帮你把id转成embedding
        attention_mask=None,   #attention里的mask
        token_type_ids=None,    # [CLS]A[SEP]B[SEP] 就这个A还是B, 有的话就全1, 没有就0
        position_ids=None,     # 位置id
        head_mask=None,       # 哪个head需要被mask掉
        inputs_embeds=None,   # 可以选择不输入id,直接输入embedding
        labels=None,          # 做分类时需要的label
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        sequence_output, pooled_output=outputs[:2]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        
        outputs = (logits, pooled_output, sequence_output,)

        if labels is not None:
            
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        
        return outputs  # loss, logits, pooled_output, sequence_output

In [31]:
# 1.载入预训练模型
args={"model_name_or_path": "/content/drive/MyDrive/model/",
    "config_name": "/content/drive/MyDrive/model/",
    "tokenizer_name": "/content/drive/MyDrive/model/",
      }

config_class, tokenizer_class = MODEL_CLASSES["bert"]
model_class=BertForClassification


config = config_class.from_pretrained(
    args["config_name"],
    finetuning_task="", 
    cache_dir=None,
)
tokenizer = tokenizer_class.from_pretrained(
    args["tokenizer_name"],
    do_lower_case=True,
    cache_dir=None,
)
model = model_class.from_pretrained(
    args["model_name_or_path"],
    from_tf=bool(".ckpt" in args["model_name_or_path"]),
    config=config,
    cache_dir=None,
)


model.to("cuda")

Some weights of the model checkpoint at /content/drive/MyDrive/model/ were not used when initializing BertForClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForClassification were not initialized from the model checkpoint at /content/drive/MyDr

BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [32]:
# 2.准备训练数据
train_features = convert_examples_to_features(train_InputExamples,
                                              tokenizer, 
                                              label_list=my_label_list, 
                                              output_mode="classification", 
                                              max_length=MAX_SEQ_LENGTH )

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [ ]:
# 看下长啥样
print("Sentence-1: ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].attention_mask)
print("-"*30)
print("Token type IDS : ", train_features[0].token_type_ids)
print("-"*30)
print("Label : ", train_features[0].label)
print("-"*30)
print("-"*30)
print("Sentence-2: ", train_InputExamples.iloc[1].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[1].text_a))
print("-"*30)
print("Input IDs : ", train_features[1].input_ids)
print("-"*30)
print("Input Masks : ", train_features[1].attention_mask)
print("-"*30)
print("Token type IDS : ", train_features[1].token_type_ids)
print("-"*30)
print("Label : ", train_features[1].label)
print("-"*30)

Sentence-1:  Does every mask protect against the coronavirus Experts say no The new coronavirus that originated in the Chinese city of Wuhan and has already killed 304 people has motivated the World Health Organisation WHO to enact an international health emergency The number of infected cases is increasing every day and have already spread to several European countries In Portugal the first two suspicions were not confirmed but the alert is global and the general panic Global health authorities have alerted citizens to take measures to prevent the spread of the virus the rate of spread of which is still unknown The use of masks is one of them mainly in China where they were already daily used due to the high signs of pollution However excessive demand from the coronavirus has led to the stock of masks depleting somewhat around the world This has led some people toimprovise using improbable objects such a s brassieres plastic bags containers and even lettuce leaves and orange peels to 

In [33]:
input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
attention_mask = torch.tensor([f.attention_mask for f in train_features], dtype=torch.long)
token_type_ids = torch.tensor([f.token_type_ids for f in train_features], dtype=torch.long)
the_labels = torch.tensor([f.label for f in train_features], dtype=torch.long)

# 这里一步每个东西都是要传到BERT模型的forward里面的, 要传哪些自己准备好
dataset = TensorDataset(input_ids, attention_mask, token_type_ids, the_labels)

In [35]:
# 3.定义训练函数
def train(train_dataset,model,tokenizer):
    no_decay=["bias","LayerNorm.weight"]
    optimizer_grouped_parameters=[
        {
            "params":[p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay":0.0,

        },
        {
            "params": [p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay":0.0
        },
    ]

    
    t_total=len(train_dataset)// 5
    optimizer=AdamW(optimizer_grouped_parameters,lr=2e-5,eps=1e-8)
    # bert里的小技巧, bert里的learning rate是不断变化的,先往上升,再往下降,这个scheduler就是用来设置这个
    scheduler=get_linear_schedule_with_warmup(
        optimizer,num_warmup_steps=0,num_training_steps=t_total
        )
    # *********************
    logger.info("*****Running training*****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", 5)


    epochs_trained=0
    global_step=0
    steps_trained_in_current_epoch=0

    tr_loss,logging_loss=0.0,0.0
    model.zero_grad()
    train_iterator=trange(epochs_trained,5,desc="Epoch",disable=False)


    for k in train_iterator: #共5个epoch
    
        #随机打包
        train_sampler=RandomSampler(train_dataset)
        train_dataloader=DataLoader(train_dataset,sampler=train_sampler,batch_size=16)
        epoch_iterator=tqdm(train_dataloader,desc="Iteration",disable=False)

        for step,batch in enumerate(epoch_iterator): 
            if steps_trained_in_current_epoch>0:
                steps_traned_in_current_epoch-=1
                continue

            model.train()
            batch=tuple(t.to("cuda") for t in batch)
            
            # 每个batch里是 input_ids, attention_mask, token_type_ids, the_labels
            # 所以传入模型时,每个参数位置对应好放进去.
            inputs={"input_ids": batch[0],"attention_mask": batch[1],"token_type_ids": batch[2], "labels": batch[3]}
   
            outputs = model(**inputs)
            loss=outputs[0]
 
            loss.backward()

            tr_loss+=loss.item()
            if (step+1)%1==0:
                torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)

                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step+=1
        logger.info("average loss:" +str(tr_loss/global_step))

    return global_step,tr_loss/global_step

In [36]:
# 4.训练
train(dataset,model,tokenizer)

Epoch: 100%|██████████| 5/5 [38:34<00:00, 462.86s/it]


(2055, 0.1880896187061104)

In [37]:
# 5.保存训练好的模型参数
import os
model.save_pretrained("/content/drive/My Drive/classification_models_1/")
tokenizer.save_pretrained("/content/drive/My Drive/classification_models_1/")

torch.save(args,os.path.join("/content/drive/My Drive/classification_models_1/","training_args.bin"))

评估部分, 载入训练好的模型, 准备评估数据, 评估

In [38]:
# 1. 载入训练好的模型
args_eval={"model_name_or_path": "/content/drive/My Drive/classification_models_1/",
    "config_name": "/content/drive/My Drive/classification_models_1/",
    "tokenizer_name": "/content/drive/My Drive/classification_models_1/",
      }

config_class, tokenizer_class = MODEL_CLASSES["bert"]
model_class=BertForClassification


config = config_class.from_pretrained(
    args_eval["config_name"],
    finetuning_task="", 
    cache_dir=None,
)
tokenizer = tokenizer_class.from_pretrained(
    args_eval["tokenizer_name"],
    do_lower_case=True,
    cache_dir=None,
)
model = model_class.from_pretrained(
    args_eval["model_name_or_path"],
    from_tf=bool(".ckpt" in args_eval["model_name_or_path"]),
    config=config,
    cache_dir=None,
)


model.to("cuda")

BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [39]:
# 2.定义评估函数函数
from sklearn.metrics import f1_score

def simple_accuracy(preds, labels):
    return (preds == labels).mean()
def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }




def evaluate(model, tokenizer, eval_dataset):


    logger.info("***** Running evaluation  *****")
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", 16)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None

    eval_sampler =RandomSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=16)

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to("cuda") for t in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps

    preds = np.argmax(preds, axis=1)

    results = simple_accuracy(preds, out_label_ids)




    return results,eval_loss

In [40]:
# 3.生成评估数据
val_features = convert_examples_to_features(val_InputExamples, 
                                            tokenizer, 
                                            label_list=my_label_list, 
                                            output_mode="classification",  
                                            max_length=MAX_SEQ_LENGTH )


val_input_ids = torch.tensor([f.input_ids for f in val_features], dtype=torch.long)
val_attention_mask = torch.tensor([f.attention_mask for f in val_features], dtype=torch.long)
val_token_type_ids = torch.tensor([f.token_type_ids for f in val_features], dtype=torch.long)
val_the_labels = torch.tensor([f.label for f in val_features], dtype=torch.long)


eval_dataset = TensorDataset(val_input_ids, val_attention_mask, val_token_type_ids, val_the_labels)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [41]:
# 4. 评估结果
results,eval_loss = evaluate(model, tokenizer, eval_dataset)
print()
print("Accuracy: ",results, "Loss: ",eval_loss)

Evaluating: 100%|██████████| 105/105 [00:41<00:00,  2.50it/s]


Accuracy:  0.8880239520958084 Loss:  0.3891933720248441


0.891156462585034

这里是LSTM开始, 首先获得embedding, 并把embedding和label组合起来

In [42]:
args_eval={"model_name_or_path": "/content/drive/My Drive/classification_models_1/",
    "config_name": "/content/drive/My Drive/classification_models_1/",
    "tokenizer_name": "/content/drive/My Drive/classification_models_1/",
      }

config_class, tokenizer_class = MODEL_CLASSES["bert"]
model_class=BertForClassification


config = config_class.from_pretrained(
    args_eval["config_name"],
    finetuning_task="", 
    cache_dir=None,
)
tokenizer = tokenizer_class.from_pretrained(
    args_eval["tokenizer_name"],
    do_lower_case=True,
    cache_dir=None,
)
model = model_class.from_pretrained(
    args_eval["model_name_or_path"],
    from_tf=bool(".ckpt" in args_eval["model_name_or_path"]),
    config=config,
    cache_dir=None,
)


model.to("cuda")

BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [43]:
train_features = convert_examples_to_features(train_InputExamples,
                                              tokenizer, 
                                              label_list=my_label_list, 
                                              output_mode="classification", 
                                              max_length=MAX_SEQ_LENGTH )

val_features = convert_examples_to_features(val_InputExamples, 
                                            tokenizer, 
                                            label_list=my_label_list, 
                                            output_mode="classification",  
                                            max_length=MAX_SEQ_LENGTH )

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [44]:
train_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
train_attention_mask = torch.tensor([f.attention_mask for f in train_features], dtype=torch.long)
train_token_type_ids = torch.tensor([f.token_type_ids for f in train_features], dtype=torch.long)
train_the_labels = torch.tensor([f.label for f in train_features], dtype=torch.long)

# 这里一步每个东西都是要传到BERT模型的forward里面的, 要传哪些自己准备好
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_token_type_ids, train_the_labels)

In [45]:
val_input_ids = torch.tensor([f.input_ids for f in val_features], dtype=torch.long)
val_attention_mask = torch.tensor([f.attention_mask for f in val_features], dtype=torch.long)
val_token_type_ids = torch.tensor([f.token_type_ids for f in val_features], dtype=torch.long)
val_the_labels = torch.tensor([f.label for f in val_features], dtype=torch.long)

# 这里一步每个东西都是要传到BERT模型的forward里面的, 要传哪些自己准备好
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_token_type_ids, val_the_labels)

In [46]:
def get_prediction(model, tokenizer, dataset):

    logger.info("***** Running prediction  *****")
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", 16)

    pooled_outputs = None

    sampler =SequentialSampler(dataset)
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)

    for batch in tqdm(dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to("cuda") for t in batch)

        with torch.no_grad():
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[3]}
            outputs = model(**inputs)
            pooled_output = outputs[2]

            if pooled_outputs is None:
                pooled_outputs = pooled_output.detach().cpu().numpy()
            else:
                pooled_outputs = np.append(pooled_outputs, pooled_output.detach().cpu().numpy(), axis=0)

    return pooled_outputs

In [47]:
train_pooled_outputs = get_prediction(model, tokenizer, train_dataset)
train_pooled_outputs.shape

Evaluating: 100%|██████████| 206/206 [02:41<00:00,  1.27it/s]


(6570, 768)

In [48]:
val_pooled_outputs = get_prediction(model, tokenizer, val_dataset)
val_pooled_outputs.shape

Evaluating: 100%|██████████| 53/53 [00:41<00:00,  1.29it/s]


(1670, 768)

In [49]:
aux = -1
len_l = 0
train_x = {}
for l, emb in zip(index_l, train_pooled_outputs):
    if l in train_x.keys():
        train_x[l]  =np.vstack([train_x[l], emb])
    else:
        train_x[l] = [emb]

len(train_x.keys())

1760

<function __main__.train>

In [50]:
train_l_final = []
label_l_final = []
for k in train_x.keys():
    train_l_final.append(train_x[k])
    label_l_final.append(train_copy.loc[k]['label'])

df_train = pd.DataFrame({'emb': train_l_final, 'label': label_l_final, })
df_train.head()
len(df_train['emb'].to_list())

1760

In [51]:
aux = -1
len_l = 0
val_x = {}

for l, emb in zip(val_index_l, val_pooled_outputs):
    if l in val_x.keys():
        val_x[l]  =np.vstack([val_x[l], emb])
    else:
        val_x[l] = [emb]


val_l_final = []
vlabel_l_final = []
for k in val_x.keys():
    val_l_final.append(val_x[k])
    vlabel_l_final.append(val.loc[k]['label'])

df_val = pd.DataFrame({'emb': val_l_final, 'label': vlabel_l_final})
df_val.head()

,emb,label
0,"[[-0.8659874, 0.57678276, 0.52520895, 0.898683...",1
1,"[[-0.7574686, 0.6062119, 0.4787779, 0.82714516...",1
2,"[[-0.85155797, 0.629637, 0.19951962, 0.8791256...",1
3,"[[-0.85437304, 0.5999677, 0.31670427, 0.880529...",1
4,"[[-0.890469, 0.6453264, 0.5792295, 0.90006673,...",1


In [52]:
df_val, df_test = train_test_split(df_val, test_size=0.4, random_state=35)

In [53]:
df_train.shape, df_val.shape, df_test.shape

((1760, 2), (264, 2), (176, 2))

In [54]:
def train_generator(df):
    num_sequences = len(df['emb'].to_list())
    batch_size = 2
    batches_per_epoch =  880
    assert batch_size * batches_per_epoch == num_sequences
    num_features= 768



    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            #yield torch.tensor(x_train), torch.tensor(y_train,dtype=int)
            # keras版本请使用这句,不需要转torch tensor
            yield x_train, y_train

In [55]:
kk= iter(train_generator(df_train))
bbb = next(kk)
print(bbb)

(array([[[-8.22501659e-01,  6.69141650e-01,  3.03386599e-01, ...,
         -8.22105408e-01, -2.33680502e-01,  7.71933421e-02],
        [-7.59405494e-01,  6.22886419e-01,  3.32301229e-01, ...,
         -8.02241564e-01, -2.76868880e-01,  2.97956944e-01],
        [-9.05327260e-01,  5.96516311e-01,  3.01024944e-01, ...,
         -8.21408033e-01, -3.97932649e-01,  5.10607243e-01],
        [-8.09049785e-01,  5.90515673e-01,  1.21692449e-01, ...,
         -8.91155064e-01, -2.52234399e-01,  2.13246524e-01],
        [-7.61562109e-01,  6.49561405e-01, -7.85850175e-03, ...,
         -8.44603837e-01, -2.79483438e-01,  1.33829433e-02]],

       [[-8.95985246e-01,  6.62095666e-01,  2.64337450e-01, ...,
         -7.66662359e-01, -3.17198247e-01,  3.21448267e-01],
        [-9.90000000e+01, -9.90000000e+01, -9.90000000e+01, ...,
         -9.90000000e+01, -9.90000000e+01, -9.90000000e+01],
        [-9.90000000e+01, -9.90000000e+01, -9.90000000e+01, ...,
         -9.90000000e+01, -9.90000000e+01, -9.9000

In [65]:
def val_generator(df):
    num_sequences_val = len(df['emb'].to_list())
    batch_size_val = 2
    batches_per_epoch_val = 88
    print(batch_size_val)
    print(batches_per_epoch_val)
    print(num_sequences_val)
    assert batch_size_val * batches_per_epoch_val == num_sequences_val
    num_features= 768


    x_list= df['emb'].to_list()
    y_list =  df.label.to_list()
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(df['emb'].to_list()[:(b + 1) * batch_size_val][-31:], key=len))
            # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                # print("li", li)
                # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            #yield torch.tensor(x_train), torch.tensor(y_train,dtype=int)
            # keras版本请使用这句,不需要转torch tensor
            yield x_train, y_train

In [142]:
len(df_val['emb'].to_list())

264

In [57]:
import tensorflow as tf


text_input = tf.keras.Input(shape=(None,768,), dtype='float32', name='text')

# keras.layers.Masking(mask_value=0.0)是用于对值为指定值的位置进行掩蔽的操作，以忽略对应的timestep。
l_mask = tf.keras.layers.Masking(mask_value=-99.)(text_input) 

# Which we encoded in a single vector via a LSTM
encoded_text = tf.keras.layers.LSTM(100,)(l_mask)
out_dense = tf.keras.layers.Dense(30, activation='relu')(encoded_text)
# And we add a softmax classifier on top
out = tf.keras.layers.Dense(10, activation='softmax')(out_dense)
# At model instantiation, we specify the input and the output:
model = tf.keras.Model(text_input, out)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None, None, 768)]       0         
                                                                 
 masking (Masking)           (None, None, 768)         0         
                                                                 
 lstm (LSTM)                 (None, 100)               347600    
                                                                 
 dense (Dense)               (None, 30)                3030      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
Total params: 350,940
Trainable params: 350,940
Non-trainable params: 0
_________________________________________________________________


In [58]:
call_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

In [59]:
!pip install torchstat==0.0.6

In [61]:
batches_per_epoch = 880
batches_per_epoch_val = 66
model.fit_generator(train_generator(df_train), steps_per_epoch=batches_per_epoch, epochs=100,
                    validation_data=val_generator(df_val), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )




Epoch 1/100
 11/880 [..............................] - ETA: 9s - loss: 0.0159 - acc: 0.2273 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


876/880 [============================>.] - ETA: 0s - loss: 0.0368 - acc: 0.09254
66
264
880/880 [==============================] - 10s 11ms/step - loss: 0.0368 - acc: 0.0920 - val_loss: 0.2545 - val_acc: 0.0644 - lr: 9.5000e-04
Epoch 2/100
880/880 [==============================] - 9s 11ms/step - loss: 0.0374 - acc: 0.0938 - val_loss: 0.2768 - val_acc: 0.0682 - lr: 9.5000e-04
Epoch 3/100
880/880 [==============================] - 9s 11ms/step - loss: 0.0357 - acc: 0.0915 - val_loss: 0.2719 - val_acc: 0.0833 - lr: 9.5000e-04
Epoch 4/100
880/880 [==============================] - 10s 11ms/step - loss: 0.0347 - acc: 0.0920 - val_loss: 0.2763 - val_acc: 0.0644 - lr: 9.5000e-04
Epoch 5/100
880/880 [==============================] - 9s 10ms/step - loss: 0.0351 - acc: 0.0949 - val_loss: 0.3041 - val_acc: 0.0682 - lr: 9.5000e-04
Epoch 6/100
877/880 [============================>.] - ETA: 0s - loss: 0.0322 - acc: 0.0924
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0009025000152178

In [74]:
num_sequences_val = len(df_train['emb'].to_list())
batch_size_val = 10
batches_per_epoch_val = 176
assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 768
model.evaluate_generator(val_generator(df_test), steps= batched_per_epoch_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


2
88
176


[1.7413341999053955, 0.09085000306367874]

In [146]:
num_sequences_val = len(df_test['emb'].to_list())
num_sequences_val

176

# 新段落